In [9]:
%pip install -q -U tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\nir.morad\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [11]:
import tensorflow as tf


In [13]:
import os

DATA_DIR  = 'data'
for dirpath, dirnames, filenames in os.walk(DATA_DIR):
    print(f"There are {len(filenames)}  directories and {len(filenames)} images in '{dirpath}'.")

There are 0  directories and 0 images in 'data'.
There are 0  directories and 0 images in 'data\testing'.
There are 44  directories and 44 images in 'data\testing\blue'.
There are 12  directories and 12 images in 'data\testing\brown'.
There are 10  directories and 10 images in 'data\testing\green'.
There are 16  directories and 16 images in 'data\testing\orange'.
There are 35  directories and 35 images in 'data\testing\yellow'.
There are 0  directories and 0 images in 'data\traning'.
There are 1046  directories and 1046 images in 'data\traning\blue'.
There are 973  directories and 973 images in 'data\traning\brown'.
There are 1050  directories and 1050 images in 'data\traning\green'.
There are 1049  directories and 1049 images in 'data\traning\orange'.
There are 1439  directories and 1439 images in 'data\traning\yellow'.


In [19]:

import pathlib
import numpy as np
data_dir = pathlib.Path(DATA_DIR+'/training')
class_names = np.array(sorted([ite.name for ite in data_dir.glob('*')]))
print(class_names)

['blue' 'brown' 'green' 'orange' 'yellow']
